In [ ]:
import os
import re
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel, LsiModel
import nltk
from nltk import WordNetLemmatizer
nltk.download('omw-1.4')
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [2]:
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/drugsComTrain_raw.csv")
reviews=data.review.values.tolist()
reviews=[re.sub("\'","",review)for review in reviews]
reviews=[re.sub("\d+","",review)for review in reviews]
reviews=[re.sub("\#","",review)for review in reviews]
reviews=[re.sub("\&","",review)for review in reviews]
reviews=[re.sub("\;","",review)for review in reviews]
reviews=[re.sub("\%","",review)for review in reviews]
reviews=[re.sub("\(","",review)for review in reviews]
reviews=[re.sub("\)","",review)for review in reviews]
reviews=[re.sub("\@","",review)for review in reviews]
def getwords(sentences):
  for s in sentences:
    yield(gensim.utils.simple_preprocess(str(s), deacc=True))
words=list(getwords(reviews))

In [3]:
def no_stopwords(wordds):
  return [[word for word in simple_preprocess(str(w).lower()) if word not in stop_words] for w in wordds]
def lemmatization(no_stop_words):
  lemmatizer = WordNetLemmatizer()
  Pos_tag = {"J": wordnet.ADJ,"N": wordnet.NOUN,"V": wordnet.VERB,"R": wordnet.ADV}
  Le_words=[]
  for w in no_stop_words:
    Le_words.append(list(lemmatizer.lemmatize(word,Pos_tag.get(nltk.pos_tag([word])[0][1][0].upper(),wordnet.NOUN)) 
    for word in simple_preprocess(str(w).lower())))
  return Le_words

In [ ]:
no_stop_words=no_stopwords(words)
lemmatized_data=lemmatization(no_stop_words)
lemmatized_data[:1]

In [ ]:
dictionary=corpora.Dictionary(lemmatized_data)
document_term_matrix=[dictionary.doc2bow(text) for text in lemmatized_data]
tfidf = gensim.models.TfidfModel(document_term_matrix, smartirs='ntc')
topic_count=10
#LSA Model using TF-IDF     
print("LSA with TF-IDF: ") 
print()
lsa_model = gensim.models.lsimodel.LsiModel(corpus=list(tfidf[document_term_matrix]),id2word=dictionary ,num_topics=15)
for i in range(topic_count):
  print("Topic:", i,end=" ")
  Topic_words=lsa_model.print_topics(num_topics=10,num_words=20)[i][1].split("+")
  for j in Topic_words:
    print(j.split("*")[1],end=" ")
  print()

#LDA Model using TF-IDF
print()
print("LDA with TF-IDF: ")
print()
lda_model = gensim.models.ldamodel.LdaModel(corpus=list(tfidf[document_term_matrix]),id2word=dictionary,
                                            num_topics=topic_count,random_state=100,chunksize=1000,passes=10)
for i in range(topic_count):
  print("Topic:", i,end=" ")
  Topic_words=lda_model.print_topics(num_topics=10,num_words=20)[i][1].split("+")
  for j in Topic_words:
    print(j.split("*")[1],end=" ")
  print()

In [ ]:
#LSA Model using (Bag Of Words) Document term matrix     
print("LSA with Bag of Words: ") 
print()
lsa_model_doc = gensim.models.lsimodel.LsiModel(corpus=document_term_matrix,id2word=dictionary ,num_topics=13)
print()
for i in range(topic_count):
  print("Topic:",i,end=" ")
  Topic_words=lsa_model_doc.print_topics(num_topics=10,num_words=20)[i][1].split("+") 
  for j in Topic_words:
    print(j.split("*")[1],end=" ")
  print()
#LDA Model using (Bag Of Words) Document term matrix
print()
print()
print("LDA with Bag of Words: ")
print()
lda_model_doc = gensim.models.ldamodel.LdaModel(corpus=document_term_matrix,id2word=dictionary,num_topics=15,
                                                random_state=100,chunksize=1000,passes=10)
for i in range(topic_count):
  print("Topic:",i,end=" ")
  Topic_words=lsa_model_doc.print_topics(num_topics=10,num_words=20)[i][1].split("+")
  for j in Topic_words:
    print(j.split("*")[1],end=" ")
  print()

In [ ]:
# Compute Perplexity
print('\n TF-IDF Perplexity: ', lda_model.log_perplexity(list(tfidf[document_term_matrix])))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=lemmatized_data, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n TF-IDF Coherence Score: ', coherence_lda)

In [ ]:
# Compute Perplexity
print('\n Document Perplexity: ', lda_model_doc.log_perplexity(document_term_matrix))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_doc, texts=lemmatized_data, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Document Coherence Score: ', coherence_lda)

In [10]:
# Coherence Function
def coherence_function(name,corpus,low,high,dictionary,data):
  print(name)
  coherence_values = []
  models = []
  for num_topics in range(low,high+1):
    if name[0:3]=="LDA":
      model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=dictionary,num_topics=num_topics)
    else:
      model = gensim.models.lsimodel.LsiModel(corpus=corpus,id2word=dictionary ,num_topics=topic_count)
    models.append(model)
    coherencemodel = CoherenceModel(model=model, texts=data, dictionary=dictionary, coherence='c_v')
    coherence_values.append(coherencemodel.get_coherence())
    print("Topics:", num_topics, "Coherence: ",coherence_values[-1])

In [ ]:
coherence_function("LSA_TF-IDF",list(tfidf[document_term_matrix]),1,20,dictionary,lemmatized_data)
print("-----------------------------------------------------------------------------------------")
coherence_function("LDA_TF-IDF",list(tfidf[document_term_matrix]),1,20,dictionary,lemmatized_data)
print("-----------------------------------------------------------------------------------------")
coherence_function("LSA_BOW",document_term_matrix,1,20,dictionary,lemmatized_data)
print("-----------------------------------------------------------------------------------------")
coherence_function("LDA_BOW",document_term_matrix,1,20,dictionary,lemmatized_data)